In [21]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# need this to be able to import processor module
import os
import sys
sys.path.insert(0, os.path.abspath('../modules'))
import preprocess as pp

## Notebook objectives


1. try Age, Fare, Ticket, Deck features. 
2. predict test age based on train data age medians

In [22]:
df_raw = pd.read_csv("../data/train.csv")

df = df_raw.copy()
df["Family"] = df.SibSp+df.Parch
df.Embarked.fillna("X", inplace=True)
df[['LastName','TitleFirstName']] = df.Name.str.split(',', expand=True)
df['Title'] = df.TitleFirstName.apply(lambda x: x.split('.') [0])
        
df = pp.infer_cabin_features(df, mark_missing=False)
df = df.drop(['Name', 'SibSp', "Parch", 'TitleFirstName', 'LastName', 'Cabin', 'PassengerId'], axis=1)

df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
Ticket        0
Fare          0
Embarked      0
Family        0
Title         0
Deck          0
dtype: int64

In [23]:
df.head()

,Survived,Pclass,Sex,Age,Ticket,Fare,Embarked,Family,Title,Deck
0,0,3,male,22.0,A/5 21171,7.2500,S,1,Mr,unknown
1,1,1,female,38.0,PC 17599,71.2833,C,1,Mrs,C
2,1,3,female,26.0,STON/O2. 3101282,7.9250,S,0,Miss,unknown
3,1,1,female,35.0,113803,53.1000,S,1,Mrs,C
4,0,3,male,35.0,373450,8.0500,S,0,Mr,unknown


In [27]:
num_df = df.copy()
num_df = pp.numerify_categorical_columns_0(num_df, columns=["Sex", "Embarked", "Title", "Ticket", "Deck",])
num_df.head()

,Survived,Pclass,Sex,Age,Ticket,Fare,Embarked,Family,Title,Deck
0,0,3,1,22.0,523,7.2500,2,1,11,16
1,1,1,0,38.0,596,71.2833,0,1,12,5
2,1,3,0,26.0,669,7.9250,2,0,8,16
3,1,1,0,35.0,49,53.1000,2,1,12,5
4,0,3,1,35.0,472,8.0500,2,0,11,16


(array([1, 0], dtype=int8), array([ 1,  0,  4,  2,  6,  5,  3,  7, 10]))

In [25]:
sex_arr = num_df.Sex.unique()
fam_arr = np.arange(0, 11, 1)
median_ages = np.zeros(len(sex_arr),len(fam_arr))

for sex in sex_arr:
    sex_median = num_df.loc[num_df.Sex == sex].Age.median()
    for family in np.fam_arr:
        pred_age = num_df.loc[((num_df.Family == family) & (num_df.Sex == sex))].Age.median()
        if pred_age > 0:
            median_ages[sex][family] = pred_age
            #print(f"sex {sex}, fam {family} --> age {median_ages[sex][family]}")
        else:
            median_ages[sex][family] = sex_median
            #print(f"sex {sex}, fam {family}* --> age {median_ages[sex][family]}")

median_ages

def fill_na_with_median(row):
    if row.Age > 0:
        return row.Age
    else:
        return median_ages[row.Sex][row.Family]

In [26]:
num_df.Age = num_df.apply(lambda row: fill_na_with_median(row), axis=1 )

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
num_df.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
Ticket      0
Fare        0
Cabin       0
Embarked    0
Family      0
Title       0
Deck        0
dtype: int64

In [ ]:
num_df.head()

,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,Family,Title,Deck
0,0,3,1,22.0,523,7.2500,unknown,2,1,11,16
1,1,1,0,38.0,596,71.2833,C85,0,1,12,5
2,1,3,0,26.0,669,7.9250,unknown,2,0,8,16
3,1,1,0,35.0,49,53.1000,C123,2,1,12,5
4,0,3,1,35.0,472,8.0500,unknown,2,0,11,16
